<a href="https://colab.research.google.com/github/savvyguru/TensorFlow/blob/master/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import tensorflow and helper libraries**

In [14]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

#tfds-nightly updates latest version of dataset
!pip-install -q tfds-nightly

#to import embedding vector
import tensorflow_hub as hub



/bin/bash: pip-install: command not found


Examine the contents of training set

**Get Dataset**

In [0]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


**Inspect Data**

In [16]:

train_examples_batch, train_labels_batch = next(iter(train_data.batch(5)))
train_examples_batch

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

**Transfer Learning using [Embedding Vector](https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa)**:


*   Output of hub layer is 50-dimension
*   Model: https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1



In [0]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")
hub_layer = hub.KerasLayer(embed, input_shape=[], 
                           dtype=tf.string, trainable=True)

**Build Rest of Neural Network**

In [18]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense_2 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


**Compile**


*   Use Binary Cross Entropy as problem is a classification problem



In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Train Model**

In [20]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 22s 718ms/step - loss: 0.7300 - accuracy: 0.4967 - val_loss: 0.6579 - val_accuracy: 0.5083
Epoch 2/20
30/30 [==============================] - 19s 621ms/step - loss: 0.6155 - accuracy: 0.5913 - val_loss: 0.5929 - val_accuracy: 0.5997
Epoch 3/20
30/30 [==============================] - 19s 625ms/step - loss: 0.5287 - accuracy: 0.7079 - val_loss: 0.5100 - val_accuracy: 0.7373
Epoch 4/20
30/30 [==============================] - 19s 624ms/step - loss: 0.4204 - accuracy: 0.8175 - val_loss: 0.4246 - val_accuracy: 0.8035
Epoch 5/20
30/30 [==============================] - 19s 624ms/step - loss: 0.3175 - accuracy: 0.8835 - val_loss: 0.3632 - val_accuracy: 0.8397
Epoch 6/20
30/30 [==============================] - 19s 624ms/step - loss: 0.2388 - accuracy: 0.9178 - val_loss: 0.3281 - val_accuracy: 0.8543
Epoch 7/20
30/30 [==============================] - 19s 622ms/step - loss: 0.1817 - accuracy: 0.9418 - val_loss: 0.3108 - val_accuracy: 0.8652

**Evaluate Model Accuracy**

In [21]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 5s - loss: 0.4355 - accuracy: 0.8470
loss: 0.435
accuracy: 0.847
